In [1]:
import networkx as nx
import numpy as np
import pandas as pd
from math import sqrt
import csv
import sys

In [2]:
from tqdm.notebook import tqdm

In [3]:
from pyBallMapper_Bokeh import graph_GUI, read_graph_from_list

In [4]:
from matplotlib.colors import ListedColormap
from matplotlib import cm
from matplotlib.colors import to_hex


In [5]:
from bokeh.plotting import figure, show

In [6]:
# to deal with large csv
maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.
    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True

## Read the graphs

Each graph must be rapresented by an adjecency list (space separated)  
We assume nodes are numbered from 1 to N  
  
The list of points covereb by each node is a file with N lines, each line contains the points id (space separated) 

# ALEX VS JONES

In [7]:
#Here we adopt standard colour palette
my_red_palette = cm.get_cmap(name='jet')
#my_red_palette = cm.get_cmap(name='Reds')

In [9]:
# Prepare Data

# adj lists path
GRAPH1_PATH = 'BM/alex_upto17_RAW/100_edges'
GRAPH2_PATH = 'BM/jones_upto17_SYMM/100_edges'

# point covered by each node path
GRAPH1_POINTS_PATH = 'BM/alex_upto17_RAW/100_points_covered_by_landmarks'
GRAPH2_POINTS_PATH = 'BM/jones_upto17_SYMM/100_points_covered_by_landmarks'

###########
# GRAPH 1 #
###########

# read graph
# ASSUME NODES ARE NUMBERED FROM 1 TO N
G1 = read_graph_from_list(GRAPH1_PATH, GRAPH1_POINTS_PATH, add_points_covered=True)

###########
# GRAPH 2 #
###########
#G2 = read_graph_from_list(GRAPH2_PATH, GRAPH2_POINTS_PATH, add_points_covered=True)

loading edgelist
loading points covered


0it [00:00, ?it/s]

computing coloring


  0%|          | 0/163 [00:00<?, ?it/s]

In [10]:
colors_dict = {1: [130, 131, 132, 136, 138, 140, 142, 143, 148, 149, 117, 151, 121, 155, 125],
               2: [128, 100, 101, 104, 106, 107, 108, 109, 112, 81, 115, 116, 84, 119, 88, 91, 93, 94],
               3: [13, 19, 24, 26, 27, 33, 35, 38, 40, 43, 45, 64, 65, 66, 68, 71, 72, 74, 76, 82, 85],
4: [1, 2, 4, 6, 9, 11, 12, 14, 18, 21, 29, 159, 161, 163, 44, 55, 56, 59, 60, 62],
5: [3, 5, 7, 8, 10, 15, 16, 17, 20, 22, 158, 160, 32, 162, 34, 49, 52, 54, 57, 58, 61],
6: [23, 25, 28, 31, 36, 37, 39, 41, 42, 46, 48, 49, 50, 51, 53, 63, 67, 69, 70, 75, 77, 78, 79, 80, 86, 90, 95, 103],
7: [129, 137, 147, 153, 83, 89, 92, 96, 97, 99, 102, 105, 110, 111, 113, 114, 118, 120, 122, 123, 127],
8: [133, 134, 135, 139, 141, 145, 156, 150, 152, 124, 126]}

In [12]:
# create coloring df

coloring_alex = pd.read_csv('BM/alex_upto17_RAW/alex_upto_17_RAW_colors.csv', sep=' ')
#coloring_jones = pd.read_csv('BM/jones_upto17_SYMM/jones_upto_17_MIRRORS_colors.csv', sep=' ')

In [13]:
coloring_alex.drop(columns=['N/A_1', 'table_number'], inplace=True)

In [14]:
coloring_alex['custom'] = 0

In [15]:
for color in tqdm(colors_dict):
    # get list of points in SELECTED_NODES
    POINTS_IN_SELECTED_NODES = set()
    for node in colors_dict[color]:
        POINTS_IN_SELECTED_NODES = POINTS_IN_SELECTED_NODES.union(set(G1.nodes[node]['points covered']))    
        
    coloring_alex.loc[POINTS_IN_SELECTED_NODES, 'custom'] = color/10

  0%|          | 0/8 [00:00<?, ?it/s]

/var/folders/4s/_33kvc6507v69vbr_vy9xsjc0000gq/T/ipykernel_19408/179424969.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  coloring_alex.loc[POINTS_IN_SELECTED_NODES, 'custom'] = color/10
/var/folders/4s/_33kvc6507v69vbr_vy9xsjc0000gq/T/ipykernel_19408/179424969.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  coloring_alex.loc[POINTS_IN_SELECTED_NODES, 'custom'] = color/10
/var/folders/4s/_33kvc6507v69vbr_vy9xsjc0000gq/T/ipykernel_19408/179424969.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  coloring_alex.loc[POINTS_IN_SELECTED_NODES, 'custom'] = color/10
/var/folders/4s/_33kvc6507v69vbr_vy9xsjc0000gq/T/ipykernel_19408/179424969.py:7: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  coloring_alex.loc[POI

In [14]:
doubled_points = []
for point_id in tqdm(coloring_alex.custom.values):
    doubled_points.append(point_id)
    doubled_points.append(point_id)
    
coloring_jones['custom'] = doubled_points

  0%|          | 0/9755329 [00:00<?, ?it/s]

In [16]:
%%time
# create a GUI with input our BM graph, 
# a dataframe with coloring functions (one value per point in the pointcloud)
# and a color palette
# in this case we use the pointcloud as coloring function
print('creating GUI 1')

G1 = read_graph_from_list(GRAPH1_PATH, GRAPH1_POINTS_PATH,
                          values_df=coloring_alex,
                          add_points_covered=False
                          )

my_fancy_gui_1 = graph_GUI(G1, my_red_palette, 
                         figsize=(800, 600),
                         render_iterations=2000,
                           render_seed=66,
                         coloring_variables=[])

print('creating GUI 2')



creating GUI 1
loading edgelist
loading points covered


0it [00:00, ?it/s]

computing coloring


  0%|          | 0/163 [00:00<?, ?it/s]

creating GUI 2
CPU times: user 7.67 s, sys: 877 ms, total: 8.54 s
Wall time: 8.58 s


In [17]:
my_fancy_gui_1.color_by_variable('custom')

color by variable custom 
MIN_VALUE: 0.000, MAX_VALUE: 0.800


(0, 0.8000000000000002)

In [18]:
show(my_fancy_gui_1.plot)

In [ ]:
G2 = read_graph_from_list(GRAPH2_PATH, GRAPH2_POINTS_PATH,
                          values_df=coloring_jones,
                          add_points_covered=False
                          )

my_fancy_gui_2 = graph_GUI(G2, my_red_palette, 
                         figsize=(800, 600),
                         render_iterations=2000,
                          coloring_variables=[])

loading edgelist
loading points covered


0it [00:00, ?it/s]

computing coloring


  0%|          | 0/890 [00:00<?, ?it/s]

In [ ]:
my_fancy_gui_2.color_by_variable('custom')

In [ ]:
show(my_fancy_gui_2.plot)

In [ ]:
jones_upto_15_30_signature

# JONES vs KHOV 15n

In [ ]:
#Here we adopt standard colour palette
#my_red_palette = cm.get_cmap(name='jet')
my_red_palette = cm.get_cmap(name='Reds')

In [ ]:
# Prepare Data

# adj lists path
GRAPH1_PATH = 'BM/jones_fromKhov_upto_15n_SYMM/30_edges'
GRAPH2_PATH = 'BM/khov_upto_15n_SYMM/20_edges'

# point covered by each node path
GRAPH1_POINTS_PATH = 'BM/jones_fromKhov_upto_15n_SYMM/30_points_covered_by_landmarks'
GRAPH2_POINTS_PATH = 'BM/khov_upto_15n_SYMM/20_points_covered_by_landmarks'

###########
# GRAPH 1 #
###########

# read graph
# ASSUME NODES ARE NUMBERED FROM 1 TO N
G1 = read_graph_from_list(GRAPH1_PATH, GRAPH1_POINTS_PATH, add_points_covered=True)

###########
# GRAPH 2 #
###########
G2 = read_graph_from_list(GRAPH2_PATH, GRAPH2_POINTS_PATH, add_points_covered=True)

In [ ]:
# colors_dict = {8: [295, 296, 171, 172, 237, 238, 307, 308],
#                6: [133, 166, 134, 165, 301, 302, 181, 182, 247, 248],
#                4: [97, 98, 163, 164, 91, 123, 141, 142, 92, 124, 51, 52, 281, 282, 283, 284, 189, 190],
#                2: [45, 46, 53, 54, 151, 152, 29, 30]
#               }

In [ ]:
colors_dict = {1: [133, 134, 11, 12, 141, 142, 273, 274, 147, 148, 151, 
                   152, 281, 282, 283, 284, 29, 30, 163, 164, 165, 166,
                   295, 296, 171, 172, 45, 46, 301, 302, 51, 52, 53, 54, 
                   181, 182, 307, 308, 189, 190, 91, 92, 97, 98, 237, 
                   238, 247, 248, 123, 124]
              }

In [ ]:
# create coloring df
coloring_jones = pd.read_csv('BM/jones_fromKhov_upto_15n_SYMM/Jones_fromKhov_upto_15n_MIRRORS_colors.csv', sep=' ')
coloring_jones

In [ ]:
coloring_jones['custom'] = 0

In [ ]:
for color in tqdm(colors_dict):
    # get list of points in SELECTED_NODES
    POINTS_IN_SELECTED_NODES = set()
    for node in colors_dict[color]:
        POINTS_IN_SELECTED_NODES = POINTS_IN_SELECTED_NODES.union(set(G1.nodes[node]['points covered']))    
        
    coloring_jones.loc[POINTS_IN_SELECTED_NODES, 'custom'] = color

In [ ]:
%%time
# create a GUI with input our BM graph, 
# a dataframe with coloring functions (one value per point in the pointcloud)
# and a color palette
# in this case we use the pointcloud as coloring function
print('creating GUI 1')

G1 = read_graph_from_list(GRAPH1_PATH, GRAPH1_POINTS_PATH,
                          values_df=coloring_jones[['custom', 'signature']],
                          add_points_covered=False
                          )

my_fancy_gui_1 = graph_GUI(G1, my_red_palette, 
                           figsize=(800, 600),
                           render_iterations=2000,
                           render_seed=66,
                           coloring_variables=['custom', 'signature'])

In [ ]:
my_fancy_gui_1.color_by_variable('custom')

In [ ]:
for node in my_fancy_gui_1.bokeh_graph:
    if my_fancy_gui_1.bokeh_graph.nodes[node]['custom'] != 1:
        my_fancy_gui_1.bokeh_graph.nodes[node]['color'] = 'white'
        
my_fancy_gui_1.graph_renderer.node_renderer.data_source.data['color'] = [my_fancy_gui_1.bokeh_graph.nodes[n]['color']
                                                                         for n in my_fancy_gui_1.bokeh_graph.nodes]

In [ ]:
show(my_fancy_gui_1.plot)

In [ ]:
print('creating GUI 2')

G2 = read_graph_from_list(GRAPH2_PATH, GRAPH2_POINTS_PATH,
                          values_df=coloring_jones[['custom', 'signature']],
                          add_points_covered=False
                          )

my_fancy_gui_2 = graph_GUI(G2, my_red_palette, 
                           figsize=(800, 600),
                           render_iterations=2000,
                           coloring_variables=['custom', 'signature'])

In [ ]:
my_fancy_gui_2.color_by_variable('custom', MAX_VALUE=1)

In [ ]:
# for node in my_fancy_gui_2.bokeh_graph:
#     if my_fancy_gui_2.bokeh_graph.nodes[node]['custom'] == 0:
#         my_fancy_gui_2.bokeh_graph.nodes[node]['color'] = 'white'
        
# my_fancy_gui_2.graph_renderer.node_renderer.data_source.data['color'] = [my_fancy_gui_2.bokeh_graph.nodes[n]['color']
#                                                                          for n in my_fancy_gui_2.bokeh_graph.nodes]

In [ ]:
show(my_fancy_gui_2.plot)

# JONES vs HOMFLY

In [ ]:
#Here we adopt standard colour palette
#my_red_palette = cm.get_cmap(name='jet')
my_red_palette = cm.get_cmap(name='Reds')

In [ ]:
# Prepare Data

# adj lists path
GRAPH1_PATH = 'BM/jones_upto_15_SYMM//50_edges'
GRAPH2_PATH = 'BM/homfly_upto_15_SYMM//50_edges'

# point covered by each node path
GRAPH1_POINTS_PATH = 'BM/jones_upto_15_SYMM/50_points_covered_by_landmarks'
GRAPH2_POINTS_PATH = 'BM/homfly_upto_15_SYMM/50_points_covered_by_landmarks'

###########
# GRAPH 1 #
###########

# read graph
# ASSUME NODES ARE NUMBERED FROM 1 TO N
G1 = read_graph_from_list(GRAPH1_PATH, GRAPH1_POINTS_PATH, add_points_covered=True)

###########
# GRAPH 2 #
###########
#G2 = read_graph_from_list(GRAPH2_PATH, GRAPH2_POINTS_PATH, add_points_covered=True)

In [ ]:
selected = [131, 132, 5, 6, 141, 142, 13, 14, 275, 276, 155, 156, 283, 
            284, 29, 32, 30, 31, 45, 46, 175, 176, 61, 62, 79, 80, 
            211, 212, 83, 84, 217, 218, 97, 98, 105, 106, 111, 112, 115, 116, 249, 250]

colors_dict = {1: selected,
               0: [node for node in G1.nodes if node not in selected]}

In [ ]:
# create coloring df
coloring_jones = pd.read_csv('BM/jones_upto_15_SYMM/jones_upto_15_MIRRORS_colors.csv', sep=' ')
coloring_jones

In [ ]:
coloring_HOMFLY = pd.read_csv('BM/homfly_upto_15_SYMM//HOMFLYPT_upto_15_MIRRORS_colors.csv', sep=' ')
coloring_HOMFLY

In [ ]:
(coloring_jones['signature'] == coloring_HOMFLY['signature']).all()

In [ ]:
coloring_jones['custom'] = 0

In [ ]:
for color in tqdm(colors_dict):
    # get list of points in SELECTED_NODES
    POINTS_IN_SELECTED_NODES = set()
    for node in colors_dict[color]:
        POINTS_IN_SELECTED_NODES = POINTS_IN_SELECTED_NODES.union(set(G1.nodes[node]['points covered']))    
        
    coloring_jones.loc[POINTS_IN_SELECTED_NODES, 'custom'] = color

In [ ]:
%%time
# create a GUI with input our BM graph, 
# a dataframe with coloring functions (one value per point in the pointcloud)
# and a color palette
# in this case we use the pointcloud as coloring function
print('creating GUI 1')

G1 = read_graph_from_list(GRAPH1_PATH, GRAPH1_POINTS_PATH,
                          values_df=coloring_jones[['signature', 'custom']],
                          add_points_covered=False
                          )

my_fancy_gui_1 = graph_GUI(G1, my_red_palette, 
                         figsize=(800, 600),
                         render_iterations=2500,
                         render_seed=42,
                         coloring_variables=['signature', 'custom'])

In [ ]:
my_fancy_gui_1.color_by_variable('custom')

In [ ]:
show(my_fancy_gui_1.plot)

In [ ]:
print('creating GUI 2')

G2 = read_graph_from_list(GRAPH2_PATH, GRAPH2_POINTS_PATH,
                          values_df=coloring_jones[['signature', 'custom']],
                          add_points_covered=False
                          )

my_fancy_gui_2 = graph_GUI(G2, my_red_palette, 
                         figsize=(800, 600),
                         render_iterations=2000,
                         render_seed=42,
                         coloring_variables=['signature'])

In [ ]:
my_fancy_gui_2.color_by_variable('custom')

In [ ]:
show(my_fancy_gui_2.plot)